# 04-01 - Vector Store - Vector Search

## Overview 


## Environment

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.38.0"
#r "nuget: Microsoft.SemanticKernel.Connectors.InMemory, 1.38.0-preview"
#r "nuget: DotNetEnv, 3.1.1"

using System.IO;
using System.ClientModel;
using System.ComponentModel;
using DotNetEnv;
using Microsoft.Extensions.VectorData;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Data;
using Microsoft.SemanticKernel.Embeddings;
using Microsoft.SemanticKernel.Connectors.InMemory;

string configurationFile = "../config/config.env";

Env.Load(configurationFile);

string apiKey = Env.GetString("SK_OPENAI_APIKEY");
string endpoint = Env.GetString("SK_OPENAI_ENDPOINT");
string embeddingDeployment = Env.GetString("SK_OPENAI_EMBEDDING_DEPLOYMENT");

Console.WriteLine($"Configuration loaded...");

Installed Packages DotNetEnv, 3.1.1 Microsoft.SemanticKernel, 1.38.0 Microsoft.SemanticKernel.Connectors.InMemory, 1.38.0-preview

Configuration loaded...


## Kernel


In [ ]:
#pragma warning disable SKEXP0001, SKEXP0010 

Kernel kernel = Kernel.CreateBuilder()
    .AddAzureOpenAITextEmbeddingGeneration(
        apiKey: apiKey, 
        endpoint: endpoint, 
        deploymentName: embeddingDeployment
    )
.Build();

Console.WriteLine($"Kernel created...");

Kernel created...


## Define Storage Model

In [ ]:
public sealed class DataStorageModel
{
    #pragma warning disable SKEXP0001
    [VectorStoreRecordKey]
    [TextSearchResultName]
    public Guid Key { get; init; }

    [VectorStoreRecordData]
    [TextSearchResultValue]
    public string Text { get; init; } = "Empty Record - No Data"; 

    [VectorStoreRecordVector(3072)]
    public ReadOnlyMemory<float> Embedding { get; init; }
    
    [TextSearchResultLink]
    public string Link {get; set;} = "";
    #pragma warning restore SKEXP0001
}

Console.WriteLine($"DataStorageModel defined...");

DataStorageModel defined...


## Create Vector Store

In [ ]:
#pragma warning disable SKEXP0001, SKEXP0010 

ITextEmbeddingGenerationService textEmbeddingGenerationService = kernel.GetRequiredService<ITextEmbeddingGenerationService>();

string[] textData = new string[]
{
    "The Super Sports Championship is an exhilarating physics competition where developers answer multiple questions, earning 10 points for each correct answer, with the first team to reach 50 points emerging victorious.",
    "The Super Bowl is the annual championship game of the National Football League (NFL) and one of the biggest sporting events in the United States.",
    "Semantic Kernel is a lightweight, open-source development kit that allows developers to easily integrate the latest AI models into their C#, Python, or Java codebase."
};

IList<ReadOnlyMemory<float>> vectorData = 
    await textEmbeddingGenerationService.GenerateEmbeddingsAsync(textData);

InMemoryVectorStore inMemoryStore = new InMemoryVectorStore();
string collectionName = "SportEventsKnowHow";

IVectorStoreRecordCollection<Guid, DataStorageModel> recordCollection = 
    inMemoryStore.GetCollection<Guid, DataStorageModel>(collectionName);
await recordCollection.CreateCollectionIfNotExistsAsync();

List<DataStorageModel> dataStorageRecords = textData
    .Select((text, index) => 
    {
        var key = Guid.NewGuid();
        return new DataStorageModel
        {
            Key = key,
            Text = text,
            Embedding = vectorData[index],
            Link = $"data:citation/text;{key}"
        };
    })
    .ToList();

Console.WriteLine($"DataStorageRecords created: ");
await foreach(Guid recordGuid in recordCollection.UpsertBatchAsync(dataStorageRecords))
{
    Console.WriteLine($"\t{recordGuid}");
} 

DataStorageRecords created: 
	37a1815f-0bf2-49ae-a217-cd6c0d5f69ee
	c7c7cd48-a04c-4bfa-a078-546800213d76
	123ac2af-c71c-4e68-98eb-827782321a19


## Query Vector Store

In [ ]:
IList<ReadOnlyMemory<float>> searchVector = await textEmbeddingGenerationService.GenerateEmbeddingsAsync(new string[] { 
    "What is the Super Sports Championship"
});

VectorSearchOptions vectorSearchOptions = new VectorSearchOptions(){ 
    Top = 3
};

VectorSearchResults<DataStorageModel> vectorSearchResult = await recordCollection.VectorizedSearchAsync(
    searchVector[0],
    vectorSearchOptions
);

IAsyncEnumerator<VectorSearchResult<DataStorageModel>> resultsList = vectorSearchResult.Results.GetAsyncEnumerator();
while (await resultsList.MoveNextAsync())
{
    Console.WriteLine($"Score: {resultsList.Current.Score} \nText: {resultsList.Current.Record.Text} \nCitation: {resultsList.Current.Record.Link} \n");
}

Console.WriteLine($"Search completed...");



Score: 0.7257648706436157 
Text: The Super Sports Championship is an exhilarating physics competition where developers answer multiple questions, earning 10 points for each correct answer, with the first team to reach 50 points emerging victorious. 
Citation: data:citation/text;37a1815f-0bf2-49ae-a217-cd6c0d5f69ee 

Score: 0.4804094731807709 
Text: The Super Bowl is the annual championship game of the National Football League (NFL) and one of the biggest sporting events in the United States. 
Citation: data:citation/text;c7c7cd48-a04c-4bfa-a078-546800213d76 

Score: 0.11162253469228745 
Text: Semantic Kernel is a lightweight, open-source development kit that allows developers to easily integrate the latest AI models into their C#, Python, or Java codebase. 
Citation: data:citation/text;123ac2af-c71c-4e68-98eb-827782321a19 

Search completed...
